## Settings


Using meteorological data from ERA5 dataset. Output unscaled.  
Filling nan values with nearest data point.   
Train for T time-stamps moving window and predict at $T^{th}$ time-stamp.   
initialize weights : ON

In [1]:
!pip install -qq torchsummaryX
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import torch.optim as optim
from time import time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
from time import sleep
torch.autograd.set_detect_anomaly(True)
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import sys
from torchsummaryX import summary
import matplotlib.pyplot as plt
from glob import glob
torch.cuda.is_available()
import multiprocessing as mp
import resource

## Defining the master class

### few terminologies

Kindly refer to ADAIN model figure to see right and left parts of network.

right_*, *_R* = right part of the network (example, right_timeseries_data, lstm_R1)   
left_*, *_L* = left part of the network (example, left_timeseries_data, fc_L1)

In [2]:
class ADAIN(nn.Module):
  def __init__(self, init_data):
    super(ADAIN, self).__init__()
    ##############################
    # ALL Metadata
    ##############################
    met_dim, time_window, drop_out = [i.item() for i in init_data]
    # met_dim = Number of meteorogical features
    # time_window = number of previous time-stamps to leverage for predictions
    # drop_out = drop out in each layer
    drop_out = drop_out/100 # convert percentage to decimal
    self.drop_out = drop_out

    ########## Defining the architecture ################
    ##############################
    # For Left station (target station) 
    ##############################
    self.lstm_L1 = nn.LSTM(input_size=met_dim, hidden_size=300, batch_first=True, dropout=drop_out)
    self.fc_L2 = nn.Linear(in_features = 300, out_features = 200)

    ##############################
    # For Right stations (train stations)
    ##############################
    self.fc_R1 = nn.Linear(in_features = 2, out_features = 100) # in_feature = 2 for 2D distance
    self.lstm_R1 = nn.LSTM(input_size = met_dim+1, hidden_size = 300, batch_first=True, dropout=drop_out) 
    # met_dim + 1 => meteorology_dimention + aqi_dimention
    self.fc_R2 = nn.Linear(in_features= 400, out_features = 200)
    self.fc_R3 = nn.Linear(in_features = 200, out_features = 200)

    ##############################
    # Attention part
    ##############################
    self.fcA1 = nn.Linear(in_features = 400, out_features = 200)
    self.fcA2 = nn.Linear(in_features = 200, out_features = 1)

    ##############################
    # Fusion Layer
    ##############################
    self.fc_final1 = nn.Linear(in_features = 400, out_features = 200)
    self.fc_final2 = nn.Linear(in_features = 200, out_features = 1)

  def forward_once(self, s_id, right_timeseries_data, right_static_data, left_output):
    #########################
    # This method does one forward pass over recurring part of the right part of network 
    #########################
    
    # s_id = station index in right_timeseries_data & right_static_data
    # right_timeseries_data -> shape = (-1, time_window, n_train_stations, met_dim+1)
    # right_static_data -> shape = (-1, n_train_stations, 2) # 2 for 2d distance
    # left_output -> shape = (-1, 200)
    
    # Right LSTM
    lstm_out, _ = self.lstm_R1(right_timeseries_data[:,:,s_id,:]) # out_shape = (-1, time_window, 300)
    lstm_out = lstm_out[:,-1,:] # taking last hidden state        # out_shape = (-1, 300)
    
    # Right FC
    fc_out = self.fc_R1(right_static_data[:,s_id,:])                
    fc_out = F.dropout(F.relu(fc_out), self.drop_out, training=self.training)
                                                                  # out_shape = (-1, 100)
        
    # Combine in FC
    combined_fc = torch.cat((fc_out, lstm_out), dim=1)            # out_shape = (-1, time_features+100)
    
    combined_fc = self.fc_R2(combined_fc)                         # out_shape = (-1, self.n_train_stations, 200)
    combined_fc = F.dropout(F.relu(combined_fc), self.drop_out, training=self.training)
    combined_fc = self.fc_R3(combined_fc)                         # out_shape = (-1, 200)        
    right_output = F.dropout(F.relu(combined_fc), self.drop_out, training=self.training) 
                                                                  # out_shape = (-1, 200)
    ##############################
    # Attention Forward
    ##############################
    attention = torch.cat((left_output, right_output), dim=1)     # out_shape = (-1, 400)
    attention = F.relu(self.fcA1(attention))                      # out_shape = (-1, 200)
    attention = self.fcA2(attention).view(-1,1,1)                 # out_shape = (-1, 1, 1)
    return attention, right_output.view(-1,1,200)                    # out_shape = (-1, 1, 1), (-1,1,200)

  def forward(self, left_timeseries, right_static_data, right_timeseries_data):
    ##############################
    # This method is main forward method of the class
    ##############################
    
    ##############################
    # Left forward
    ##############################
    
    # (-1,*,*)-> -1 in beginning of the shape is batch_size throughout this code
    
    # Left lstm
    lstm_out, _ = self.lstm_L1(left_timeseries)                   # out_shape = (-1, self.time_window, 300)
    lstm_out = lstm_out[:,-1,:] # taking last hidden state        # out_shape = (-1, 300)

    # Left higher FC (ommiting lower FC as we don't have POI data)
    fc_out = self.fc_L2(lstm_out)
    left_output = F.relu(fc_out)

    ##############################
    # Right & Attention forward
    ##############################
    for s_id in range(right_timeseries_data.shape[2]):
        attention, right_output = self.forward_once(s_id, right_timeseries_data, right_static_data, left_output)
    
    attention_softmax = attention/attention.sum(axis=1).view(-1,1,1) # out_shape = (-1, 20, 1)
    attention_output = (right_output * attention_softmax).sum(axis=1) # sum((-1,20,1) * (-1,20,1)) ==> (-1, 1)
    
    ##############################
    # Final fusion forward
    ##############################
    
    fusion_output = self.fc_final1(torch.cat((attention_output, left_output), dim=1)) # (-1, 200)
    fusion_output = F.dropout(F.relu(fusion_output), self.drop_out, training=self.training)
    fusion_output = self.fc_final2(fusion_output) # (-1,200)->(-1, 1)
    return fusion_output.view(-1)

### Summary

In [3]:
# model = ADAIN(init_data.to('cuda')).to('cuda')
# print(summary(model, *[i.to(device) for i in train_dataloaders[left_station][batch][:3]]))

### Let's go to training

In [4]:
batch_size = 24
met_dim = 11
n_train_stations=21
n_val_stations=3
n_test_stations=12
time_window = 24
n_folds = 3
regularizer = 0.01
lr = 0.01
device='cuda'
n_epochs = 5
drop_out = 50 # in percentage

###
t_start = 0
t_end = 24*10

# Data Preperation Fold 0

In [5]:
torch.manual_seed(0) # to control the shuffle in train data
f_n = 0

data_dict = pd.read_pickle('../Data/ERA5/train_val_test.pickle')
ignore_cols = ['name_chinese','name_english','district_id']
latlon_data = pd.read_csv('../Data/raw/Beijing_station.csv').set_index('station_id').sort_index().drop(columns=ignore_cols)
t_steps = data_dict[f_n]['train'].index.unique().sort_values()[t_start:t_end]

#######################
# Data prep
#######################
train_dataloaders = {}
val_dataloaders = {}
test_dataloaders = {}

train_data = data_dict[f_n]['train']
val_data = data_dict[f_n]['val']
test_data = data_dict[f_n]['test']

train_stations = train_data.station_id.unique().tolist()
val_stations = val_data.station_id.unique().tolist()
test_stations = test_data.station_id.unique().tolist()

##############################################
# Train data preperation
##############################################
for s_id, left_station in enumerate(train_data.station_id.unique()):
    clear_output(wait=True)
    print('preparing train data for ', s_id, left_station)
    
    right_stations = train_data.station_id.unique()
    right_stations = np.setdiff1d(right_stations, left_station)
    drop_cols = ['station_id','PM25','latitude','longitude','time']
    left_timeseries_data = train_data[train_data.station_id==left_station].drop(columns=drop_cols)
    left_aqi_data = train_data[train_data.station_id==left_station]['PM25']
    
    # Initialize empty arrays
    left_timeseries = np.zeros(((len(t_steps)-time_window+1), time_window, met_dim), dtype=np.float32)*np.nan
    left_aqi = np.zeros(((len(t_steps)-time_window+1), 1), dtype=np.float32)*np.nan
    ###################################
    
    drop_cols.remove('PM25')
    right_timeseries_data = train_data[train_data.station_id.isin(right_stations)].drop(columns=drop_cols)
    right_timeseries_data = np.stack(np.split(right_timeseries_data.values, 
                                               indices_or_sections=len(right_stations)))
    right_timeseries_data = np.moveaxis(right_timeseries_data, 0, 1)
    
    # Initialize empty arrays
    right_timeseries = np.nan * np.zeros(((len(t_steps)-time_window+1), 
                                           time_window, 
                                           len(right_stations), 
                                           met_dim+1), dtype=np.float32)
    ################################
    
    right_dis_data = (train_data[train_data.station_id.isin(right_stations)]\
    .set_index('station_id')[['latitude','longitude']].drop_duplicates().values-\
    train_data[train_data.station_id==left_station]\
    .set_index('station_id')[['latitude','longitude']].drop_duplicates().values).astype(np.float32)

    right_dis = np.zeros(((len(t_steps)-time_window+1), len(right_stations), 2), 
                             dtype=np.float32)*np.nan

    for t in range(len(t_steps)-time_window+1):
        left_timeseries[t,:,:] = left_timeseries_data[str(t_steps[t]):str(t_steps[t+time_window-1])]
        left_aqi[t,:] = left_aqi_data[str(t_steps[t+time_window-1])]
        right_timeseries[t,:,:,:] = right_timeseries_data[t:t+time_window]
        right_dis[t,:,:] = right_dis_data
    dataset = TensorDataset(torch.tensor(left_timeseries), 
                            torch.tensor(right_dis), 
                            torch.tensor(right_timeseries),
                            torch.tensor(left_aqi))
    train_dataloaders[left_station] = list(DataLoader(dataset, batch_size=batch_size, shuffle=True))

    
##############################################
# Validation data preperation
##############################################
for s_id, left_station in enumerate(val_stations):
    clear_output(wait=True)
    print('preparing val data for ', s_id, left_station)
    drop_cols = ['station_id','PM25','latitude','longitude','time']
    left_timeseries_data = val_data[val_data.station_id==left_station].drop(columns=drop_cols)
    left_aqi_data = val_data[val_data.station_id==left_station]['PM25']
    
    left_timeseries = np.zeros(((len(t_steps)-time_window+1), time_window, met_dim), dtype=np.float32)*np.nan
    left_aqi = np.zeros(((len(t_steps)-time_window+1), 1), dtype=np.float32)*np.nan

    drop_cols.remove('PM25')
    right_timeseries_data = train_data.drop(columns=drop_cols)
    right_timeseries_data = np.stack(np.split(right_timeseries_data.values, 
                                               indices_or_sections=len(right_stations)))
    right_timeseries_data = np.moveaxis(right_timeseries_data, 0, 1)
    right_timeseries = np.nan * np.zeros(((len(t_steps)-time_window+1), 
                                           time_window, 
                                           len(right_stations), 
                                           met_dim+1), dtype=np.float32)

    right_dis_data = (train_data\
    .set_index('station_id')[['latitude','longitude']].drop_duplicates().values-\
    val_data[val_data.station_id==left_station]\
    .set_index('station_id')[['latitude','longitude']].drop_duplicates().values).astype(np.float32)

    right_dis = np.zeros(((len(t_steps)-time_window+1), len(train_stations), 2), 
                             dtype=np.float32)*np.nan

    for t in range(len(t_steps)-time_window+1):
        left_timeseries[t,:,:] = left_timeseries_data[str(t_steps[t]):str(t_steps[t+time_window-1])]
        left_aqi[t,:] = left_aqi_data[str(t_steps[t+time_window-1])]
        right_timeseries[t,:,:,:] = right_timeseries_data[t:t+time_window]
        right_dis[t,:,:] = right_dis_data
    dataset = TensorDataset(torch.tensor(left_timeseries), 
                            torch.tensor(right_dis), 
                            torch.tensor(right_timeseries),
                            torch.tensor(left_aqi))
    val_dataloaders[left_station] = list(DataLoader(dataset, batch_size=batch_size))

##############################################
# Test data preperation
##############################################
for s_id, left_station in enumerate(test_stations):
    clear_output(wait=True)
    print('preparing test data for ', s_id, left_station)
    drop_cols = ['station_id','PM25','latitude','longitude','time']
    left_timeseries_data = test_data[test_data.station_id==left_station].drop(columns=drop_cols)
    left_aqi_data = test_data[test_data.station_id==left_station]['PM25']
    
    left_timeseries = np.zeros(((len(t_steps)-time_window+1), time_window, met_dim), dtype=np.float32)*np.nan
    left_aqi = np.zeros(((len(t_steps)-time_window+1), 1), dtype=np.float32)*np.nan

    drop_cols.remove('PM25')
    right_timeseries_data = train_data.drop(columns=drop_cols)
    right_timeseries_data = np.stack(np.split(right_timeseries_data.values, 
                                               indices_or_sections=len(right_stations)))
    right_timeseries_data = np.moveaxis(right_timeseries_data, 0, 1)
    right_timeseries = np.nan * np.zeros(((len(t_steps)-time_window+1), 
                                           time_window, 
                                           len(right_stations), 
                                           met_dim+1), dtype=np.float32)

    right_dis_data = (train_data\
    .set_index('station_id')[['latitude','longitude']].drop_duplicates().values-\
    test_data[test_data.station_id==left_station]\
    .set_index('station_id')[['latitude','longitude']].drop_duplicates().values).astype(np.float32)

    right_dis = np.zeros(((len(t_steps)-time_window+1), len(train_stations), 2), 
                             dtype=np.float32)*np.nan

    for t in range(len(t_steps)-time_window+1):
        left_timeseries[t,:,:] = left_timeseries_data[str(t_steps[t]):str(t_steps[t+time_window-1])]
        left_aqi[t,:] = left_aqi_data[str(t_steps[t+time_window-1])]
        right_timeseries[t,:,:,:] = right_timeseries_data[t:t+time_window]
        right_dis[t,:,:] = right_dis_data
    dataset = TensorDataset(torch.tensor(left_timeseries), 
                            torch.tensor(right_dis), 
                            torch.tensor(right_timeseries),
                            torch.tensor(left_aqi))
    test_dataloaders[left_station] = list(DataLoader(dataset, batch_size=batch_size))

preparing test data for  11 1036.0


## Training: Fold 0

In [ ]:
#######################
# Model definition
#######################
init = time()
torch.manual_seed(0)
global_loss = np.inf
init_data = torch.tensor([met_dim, time_window, drop_out])
ADAINmodel = ADAIN(init_data).to(device)
criterion = nn.MSELoss()
val_criterion = nn.MSELoss()
ADAINoptimizer = optim.Adam(ADAINmodel.parameters(), lr=lr, weight_decay=regularizer)
# ADAINoptimizer = optim.SGD(ADAINmodel.parameters(), lr=lr, momentum=0.9)

# Initialize weights
def weights_init_uniform(model):
    for p in model.parameters():
        torch.nn.init.uniform_(p, -0.1, 0.1)
weights_init_uniform(ADAINmodel)

### Train + Validation #############

for epoch in range(n_epochs):
    #################################
    # Training
    #################################
    ADAINmodel.train() # Enable train mode

    out = []
    grn = []
    n_batch = len(train_dataloaders[list(train_dataloaders.keys())[0]])
    for batch in range(n_batch):
        ADAINoptimizer.zero_grad()
        for left_station in train_data.station_id.unique():
            # forward + backward + optimize
            outputs_t = ADAINmodel(*[i.to(device) for i in train_dataloaders[left_station][batch][:3]])
            loss = criterion(outputs_t, train_dataloaders[left_station][batch][3].to(device))
            out.append(outputs_t.cpu().detach())
            grn.append(train_dataloaders[left_station][batch][3])
            loss.backward()
            clear_output(wait=True)
            print('train','f_n=',f_n,'epoch=', epoch, left_station, 'loss=',loss.item(), 
                  'batch=',batch,'of',n_batch, 'out',out[-1],'grn',grn[-1])
        ADAINoptimizer.step()
    print('Training over for epoch',epoch)
    pd.to_pickle({'fold'+str(f_n): {'out':np.concatenate(out).squeeze(), 'grn':np.concatenate(grn).squeeze()}}, 
                 '../Results/test_ADAIN_fold_'+str(f_n)+'values.pickle')
    pd.to_pickle(np.sqrt(np.mean((np.concatenate(out).squeeze()- np.concatenate(grn).squeeze())**2))
                 , '../Results/'+str(epoch).zfill(4)+'trn_ADAIN_'+str(f_n)+'_fold.pickle')

    #################################
    # Validation
    #################################
    ADAINmodel.eval()
    print('started validation',epoch)
    out = []
    grn = []
    for batch in range(len(val_dataloaders[list(val_dataloaders.keys())[0]])):
        for val_station in val_data.station_id.unique():
            outputs_v = ADAINmodel(*[i.to(device) for i in val_dataloaders[val_station][batch][:3]])
            out.append(outputs_v.cpu().detach())
            grn.append(val_dataloaders[val_station][batch][3])
            loss = criterion(outputs_v, val_dataloaders[val_station][batch][3].to(device))
            clear_output(wait=True)
            print('f_n',f_n, epoch, val_station, loss.item(), batch)
    epoch_loss = np.sqrt(np.mean((np.concatenate(out).squeeze()- np.concatenate(grn).squeeze())**2))
    if epoch_loss < global_loss:
        global_loss = epoch_loss
        torch.save(ADAINmodel.state_dict(), '../models/'+str(f_n)+'fold_ADAIN.h5')
    pd.to_pickle(epoch_loss, '../Results/'+str(epoch).zfill(4)+'val_ADAIN_'+str(f_n)+'_fold.pickle')

#################################
# Testing
#################################
test_model = ADAIN(init_data).to(device)
test_model.load_state_dict(torch.load('../models/'+str(f_n)+'fold_ADAIN.h5'))
test_model.eval()
print('started test')
out = []
grn = []
for batch in range(len(test_dataloaders[list(test_dataloaders.keys())[0]])):
    for test_station in test_data.station_id.unique():
        outputs_v = test_model(*[i.to(device) for i in test_dataloaders[test_station][batch][:3]])
        out.append(outputs_v.cpu().detach())
        grn.append(val_dataloaders[val_station][batch][3])
        loss = criterion(outputs_v, test_dataloaders[test_station][batch][3].to(device))
        clear_output(wait=True)
        print('f_n',f_n,test_station, loss.item(), batch)
pd.to_pickle({'fold'+str(f_n): {'out':np.concatenate(out).squeeze(), 'grn':np.concatenate(grn).squeeze()}}, '../Results/test_ADAIN_fold_'+str(f_n)+'values.pickle')
pd.to_pickle(np.sqrt(np.mean((np.concatenate(out).squeeze()- np.concatenate(grn).squeeze())**2)), '../Results/test_ADAIN_fold'+str(f_n)+'.pickle')
print(time()-init, 'seconds')

train f_n= 0 epoch= 0 1001.0 loss= 609.156494140625 batch= 8 of 10 out tensor([29.0463, 44.1916, 43.3254, 65.4282, 51.3721, 37.2432, 35.4954, 49.4398,
        48.9868, 45.2009, 48.6309, 54.2246, 44.1604, 36.6907, 52.0210, 51.1292,
        41.3338, 50.1381, 39.6441, 38.7214, 45.9214, 46.6660, 44.6516, 47.4464]) grn tensor([[ 65.],
        [ 31.],
        [ 29.],
        [ 75.],
        [ 62.],
        [ 34.],
        [ 90.],
        [ 62.],
        [ 19.],
        [ 26.],
        [ 29.],
        [ 77.],
        [ 36.],
        [ 61.],
        [ 62.],
        [105.],
        [ 57.],
        [ 77.],
        [ 57.],
        [ 62.],
        [ 57.],
        [ 37.],
        [ 20.],
        [ 54.]])


In [14]:
## Train loss
for i in range(n_epochs):
    print(pd.read_pickle('../Results/'+str(i).zfill(4)+'trn_ADAIN_'+str(f_n)+'_fold.pickle'))

40.564587
32.483482
31.850136
31.705538
31.827078


In [15]:
## Validation loss
for i in range(n_epochs):
    print(pd.read_pickle('../Results/'+str(i).zfill(4)+'val_ADAIN_'+str(f_n)+'_fold.pickle'))

24.826324
24.656431
23.86893
24.47673
21.965303


In [16]:
## Test loss
pd.read_pickle('../Results/test_ADAIN_fold'+str(f_n)+'.pickle')

29.044458

## Manual Checking

In [17]:
ind = 56
# Train

# val

# Test

grn, out


[(62.0, 50.62975311279297),
 (70.0, 50.589359283447266),
 (72.0, 50.215179443359375),
 (65.0, 49.79447937011719),
 (62.0, 49.249534606933594),
 (53.0, 48.84075164794922),
 (54.0, 48.80052947998047),
 (59.0, 48.88352584838867),
 (55.0, 49.04583740234375),
 (56.0, 49.50800323486328),
 (63.0, 49.819602966308594),
 (66.0, 49.43227767944336),
 (74.0, 49.9012565612793),
 (62.0, 50.15349578857422),
 (61.0, 50.17626953125),
 (60.0, 50.11144256591797),
 (40.0, 49.770362854003906),
 (28.0, 49.849403381347656),
 (31.0, 50.25349807739258),
 (34.0, 50.52042007446289),
 (34.0, 50.44000244140625),
 (39.0, 50.47332000732422),
 (44.0, 50.410545349121094),
 (49.0, 50.12386703491211)]

# AUTHOR RMSE IS 49.45